## Overview

The COVID-19 pandemic has had a profound and long-lasting impact on the global economy; therefore, household credit card debt is also influenced. This project aims to investigate how financial literacy and household credit card debt have changed after COVID-19. Our team will be using two large-scale datasets from the years 2020 and 2024 Federal Reserve’s Survey of Household Economics and Decisionmaking (SHED).

By comparing the 2020 and 2024 SHED datasets, this project will explore how levels of financial literacy, credit access, and repayment behavior have evolved in response to the pandemic’s lasting economic effects. The analysis will examine demographic variables such as income, age, and education level to identify which groups experienced the most significant changes in credit card usage and debt management. We will also consider economic influences to interpret shifts in household financial behavior.

Our approach combines data integration, cleaning, and statistical analysis to ensure a transparent and reproducible workflow. The findings will provide insights into how households adapted their financial strategies in the years following COVID-19 and last year of 2024 after COVID-19.  

The CFPB of Consumer Credit Trends dataset provides monthly, nationwide information on new credit card originations. It includes the total dollar volume of new credit card limits issued to different credit score groups, ranging from deep subprime to super-prime borrowers. The dataset contains both seasonally adjusted and unadjusted lending volumes, allowing analysis of raw lending activity as well as smoothed long-term trends from the year 2007 Jan to 2025 Apr. Each row represents one credit score category for a given month. This dataset helps us understand how lending patterns differ across borrower risk levels.

This dataset will allow us to enrich SHED respondents’ financial experiences with contextual information about the broader credit environment. Adding the CFPB dataset introduces some challenges in the data integration process, which strengthens the methodological rigor of our project.


The overall goal is to produce a reproducible, data-integrated analysis that would demonstrate how the pandemic and economic disruptions affect individuals’ financial decision-making and credit outcomes. Through the process of merging these two large federal datasets, we will illustrate multiple aspects of the data lifecycle and other techniques that were learned in class.

In addition to the technical aims, this project will contribute to a broader understanding of financial resilience: how knowledge, education, and access to certain financial resources influence households to make financial decisions and manage debt in a time of crisis. And by highlighting the difference between 2020 and 2024, we hope to address the importance of promoting household stability in an unstable economic setting.

## Research Question

1. Did the strength of the relationship between financial literacy and household credit card debt change after the COVID-19 pandemic?

2. Did households with higher financial literacy show smaller increases in debt post-COVID-19?



In [ ]:
pip install pyreadstat

In [ ]:
from pathlib import Path
import pyreadstat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import hashlib
from pathlib import Path
import io

pd.set_option('display.max_rows', None)

For importing **pyreadstat**, please download **pip install pyreadstat**.

# Collect / Acquire

We will collect the dataset:

1. SHED 2020 and 2024 : https://www.federalreserve.gov/consumerscommunities/shed_data.htm

- Source : Board of Governors of the Federal Reserve System

- Format : SAS file of **publicxxxx.sas7dbat**

- Dataset : https://www.federalreserve.gov/consumerscommunities/files/SHED_public_use_data_2020_(SAS).zip (2020), https://www.federalreserve.gov/consumerscommunities/files/SHED_public_use_data_2024_(SAS).zip (2024)

- Action : Download SAS file and documentation, converting the SAS file to CSV with analyzing its documentation.

- Documentation :
  -  https://www.federalreserve.gov/consumerscommunities/files/SHED_2024codebook.pdf (2024)

  - https://www.federalreserve.gov/consumerscommunities/files/SHED_2020codebook.pdf (2020)

2. CFPB : https://www.consumerfinance.gov/data-research/consumer-credit-trends/credit-cards/borrower-risk-profiles/#anchor_lending-levels

- Source : CFPB Consumer Credit Trends of borrower risk profiles (Lending Levels)

- Format : CSV

- Dataset : https://files.consumerfinance.gov/data/consumer-credit-trends/credit-cards/volume_data_Score_Level_CRC.csv

- Action : Download and save as raw CSV for later aggregation and integration.

In [ ]:
# read the SAS file for SHED 2024
df_shed24, meta = pyreadstat.read_sas7bdat("/content/shed2024.sas7bdat")

# convert to csv
df_shed24.to_csv("shed2024.csv", index=False)

row_df_shed24 = df_shed24.shape[0]

print(df_shed24.shape)
df_shed24.head()

(12295, 753)


,CaseID,caseid2023,caseid2022,duration,weight,weight_pop,panel_weight,panel_weight_pop,x2023,x2022,...,E4_d_iflag,E4_e_iflag,E4_f_iflag,CH2A_iflag,race_5cat,inc_4cat_50k,educ_4cat,pay_casheqv,atleast_okay,control
0,1.0,1.0,1.0,1333.0,0.6149,12971.9817,1.2153,78509.7641,Yes,Yes,...,0.0,0.0,0.0,0.0,White,"$25,000â$49,999",Bachelor's degree or more,Yes,Yes,Public
1,2.0,NaN,2.0,975.0,0.8301,17511.2242,NaN,NaN,No,Yes,...,0.0,0.0,0.0,0.0,White,"$100,000 or more",Bachelor's degree or more,Yes,Yes,Public
2,3.0,7.0,NaN,2146.0,0.4447,9381.0980,0.8061,52078.4957,Yes,No,...,0.0,0.0,0.0,0.0,White,"$50,000â$99,999",Bachelor's degree or more,Yes,Yes,
3,4.0,NaN,NaN,1214.0,0.8333,17580.0326,NaN,NaN,No,No,...,0.0,0.0,0.0,0.0,White,"$50,000â$99,999",High school degree or GED,Yes,Yes,
4,5.0,NaN,9.0,1168.0,0.9064,19121.3763,NaN,NaN,No,Yes,...,0.0,0.0,0.0,0.0,White,"$50,000â$99,999",Some college/technical or associates degree,Yes,Yes,Public


In [ ]:
# read the SAS file for SHED 2020
df_shed20, meta = pyreadstat.read_sas7bdat("/content/public2020.sas7bdat")

# convert to csv
df_shed20.to_csv("shed2020.csv", index=False)

print(df_shed20.shape)
df_shed20.head()

(11648, 372)


,CASEID,caseid2019,caseid2018,DURATION,WEIGHT,weight_pop,weight_july2020_panel,weight_july2020_panel_pop,weight_2019_panel,weight_2019_panel_pop,...,ACTCAT,CFPB_score,race_5cat,educ_4cat,inc_4cat_50k,atleast_okay,pay_casheqv,AFS,skip_medical,DOV_FL
0,1.0,NaN,60.0,542.0,0.9829,21260.3220,NaN,NaN,NaN,NaN,...,NaN,62.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0
1,2.0,NaN,NaN,1357.0,0.6080,13151.6255,NaN,NaN,NaN,NaN,...,2.0,72.0,1.0,4.0,2.0,1.0,1.0,0.0,0.0,0.0
2,3.0,NaN,NaN,1607.0,0.6599,14274.1602,NaN,NaN,NaN,NaN,...,1.0,62.0,1.0,3.0,3.0,1.0,1.0,0.0,0.0,1.0
3,4.0,NaN,NaN,680.0,1.1181,24185.3717,NaN,NaN,NaN,NaN,...,2.0,58.0,1.0,4.0,-1.0,1.0,1.0,0.0,0.0,1.0
4,5.0,9.0,NaN,2287.0,0.5406,11693.6698,0.9723,130565.731,0.5031,33391.1165,...,1.0,90.0,1.0,4.0,4.0,1.0,1.0,0.0,0.0,0.0


In [ ]:
# read CFPB dataset
df_crc = pd.read_csv("/content/volume_data_Score_Level_CRC (4).csv")

print(df_crc.shape)
df_crc.head()


(1100, 5)


,month,date,vol,vol_unadj,credit_score_group
0,84,2007-01,2.544778e+10,2.261068e+10,Super-prime
1,84,2007-01,2.566262e+09,2.139431e+09,Near-prime
2,84,2007-01,4.680974e+08,4.052789e+08,Deep subprime
3,84,2007-01,8.426960e+09,7.433527e+09,Prime
4,84,2007-01,8.445496e+08,7.369389e+08,Subprime


We will explore the SHED datasets of 2020 and 2024 of the behavior of financial literacy and household credit first focusing on our research questions. Then, we will aggregate the CFPB dataset later on for more information about the credit risk.

# Data Quality





In [ ]:
print("SHED 2024 rows and columns : ", df_shed24.shape)
print("SHED 2020 rows and columns : ", df_shed20.shape)



SHED 2024 rows and columns :  (12295, 753)
SHED 2020 rows and columns :  (11648, 372)


For SHED dataset, each row represents each person participated in the each columns of survey questions. In 2024, 12295 people participated with 753 survey questions. In 2020, 11648 people participated with 372 survey questions.

Considering the fitness for use, we want to explore the dataset and answer our research questions of:

1. Did the strength of the relationship between financial literacy and household credit card debt change after the COVID-19 pandemic?

2. Did households with higher financial literacy show smaller increases in debt post-COVID-19?

3. Do regional market conditions explain part of the change in debt behavior?

We want to extract the relevant features related to our research goal in order to know what we want to do with the data. We will identify the features through the documentation of SHED provided, https://www.federalreserve.gov/consumerscommunities/files/SHED_2024codebook.pdf

For the **SHED** dataset:

**credit card**
- C2A — Credit card ownership
- C4A — Frequency of carrying unpaid balance

**Financial Literacy**
- EF1 — Has emergency savings
- EF2 — Ability to cover 3 months if income is lost

EF3 - Suppose that you
have an emergency expense that costs $400.
Based on your current financial situation,
How would you pay for this expense.

- EF3_a — Put it on my credit card and pay it off in full at the next statement

- EF3_b — Put it on my credit card and pay it off over
time
- EF3_c — With the money currently in my
checking/savings account or with cash
- EF3_d — Using money from a bank loan or line of credit
- EF3_e — By borrowing from a friend or family member
- EF3_f — Using a payday loan, deposit advance, or
overdraft
- EF3_g — By selling something
- EF3_h — I wouldn't be able to pay for the expense
right now




We will go over SHED 2024 first, then 2020 dataset as well.

In [ ]:
df_shed24 = df_shed24[["C2A", "C4A","EF1","EF2","EF3_a","EF3_b","EF3_c","EF3_d",
                       "EF3_e","EF3_f","EF3_g","EF3_h"]]
df_shed24.head()

,C2A,C4A,EF1,EF2,EF3_a,EF3_b,EF3_c,EF3_d,EF3_e,EF3_f,EF3_g,EF3_h
0,Yes,Never carried an unpaid balance (always pay in...,Yes,,Yes,No,No,No,No,No,No,No
1,Yes,Never carried an unpaid balance (always pay in...,Yes,,No,No,Yes,No,No,No,No,No
2,Yes,Never carried an unpaid balance (always pay in...,No,No,No,No,Yes,No,No,No,No,No
3,Yes,Never carried an unpaid balance (always pay in...,Yes,,Yes,No,No,No,No,No,No,No
4,Yes,Never carried an unpaid balance (always pay in...,No,No,Yes,No,No,No,No,No,No,No


We will use these features to caculate the risk credit score for distributing the credit card behavior based on the survey. Then we will divide into 5 groups as:

- 0~1 : Super-prime
- 2~3 : Prime
- 4~5 : Near-prime
- 6~7 : Subprime
- 8+: Deep Subprime

The more score it has, the more risk circumstances on their paying credit card balance.

In [ ]:
df_shed24["risk_score"] = 0

# EF1, EF2: no emergency savings → risk
df_shed24.loc[df_shed24["EF1"] == "No", "risk_score"] += 1
df_shed24.loc[df_shed24["EF2"] == "No", "risk_score"] += 1

# EF3 emergency handling (risky behaviors)
df_shed24.loc[df_shed24["EF3_b"] == "Yes", "risk_score"] += 1   # credit card over time
df_shed24.loc[df_shed24["EF3_e"] == "Yes", "risk_score"] += 1   # borrow from friends/family
df_shed24.loc[df_shed24["EF3_d"] == "Yes", "risk_score"] += 1   # bank loan/LOC

# credit card access
df_shed24.loc[df_shed24["C2A"] == "No", "risk_score"] += 1     # no credit card → often subprime

# unpaid balance
df_shed24.loc[df_shed24["C4A"] =="Once", "risk_score"] += 1
df_shed24.loc[df_shed24["C4A"] =="Some of the time", "risk_score"] += 2
df_shed24.loc[df_shed24["C4A"] =="Most or all of the time", "risk_score"] += 3

df_shed24.head()

,C2A,C4A,EF1,EF2,EF3_a,EF3_b,EF3_c,EF3_d,EF3_e,EF3_f,EF3_g,EF3_h,risk_score
0,Yes,Never carried an unpaid balance (always pay in...,Yes,,Yes,No,No,No,No,No,No,No,0
1,Yes,Never carried an unpaid balance (always pay in...,Yes,,No,No,Yes,No,No,No,No,No,0
2,Yes,Never carried an unpaid balance (always pay in...,No,No,No,No,Yes,No,No,No,No,No,2
3,Yes,Never carried an unpaid balance (always pay in...,Yes,,Yes,No,No,No,No,No,No,No,0
4,Yes,Never carried an unpaid balance (always pay in...,No,No,Yes,No,No,No,No,No,No,No,2


In [ ]:
df_shed24["risk_score"].value_counts()

,count
risk_score,
0,4626
3,2035
2,1431
1,1259
4,1237
5,894
6,688
7,115
8,10


In [ ]:
def map_group(score):
    if score <= 1:
        return "Super-prime"
    elif score <= 3:
        return "Prime"
    elif score <= 5:
        return "Near-prime"
    elif score <= 7:
        return "Subprime"
    else:
        return "Deep subprime"

df_shed24["proxy_credit_group"] = df_shed24["risk_score"].apply(map_group)

In [ ]:
df_shed24.head()

,C2A,C4A,EF1,EF2,EF3_a,EF3_b,EF3_c,EF3_d,EF3_e,EF3_f,EF3_g,EF3_h,risk_score,proxy_credit_group
0,Yes,Never carried an unpaid balance (always pay in...,Yes,,Yes,No,No,No,No,No,No,No,0,Super-prime
1,Yes,Never carried an unpaid balance (always pay in...,Yes,,No,No,Yes,No,No,No,No,No,0,Super-prime
2,Yes,Never carried an unpaid balance (always pay in...,No,No,No,No,Yes,No,No,No,No,No,2,Prime
3,Yes,Never carried an unpaid balance (always pay in...,Yes,,Yes,No,No,No,No,No,No,No,0,Super-prime
4,Yes,Never carried an unpaid balance (always pay in...,No,No,Yes,No,No,No,No,No,No,No,2,Prime


Let's check if there is missing values considering its completeness.

In [ ]:
df_shed24.isna().sum()

,0
C2A,0
C4A,0
EF1,0
EF2,0
EF3_a,0
EF3_b,0
EF3_c,0
EF3_d,0
EF3_e,0
EF3_f,0


There is no missing values.

## Semantic check
We will check the error of semantic accuracy for our data quality, if there is unmatched meaning. We will use the column of **EF2** meaning they have aility to cover 3 months if income is lost. Leveraging that feature, we will compare the **EF3** series of features. It is impossible to say all "No" on **EF3** series who cannot pay the cover in any reason, even though they have a ability to cover 3 months if income is lost.

In [ ]:
cannot_pay_values = {"No"}

ef3_cols = ["EF3_a","EF3_b","EF3_c","EF3_d", "EF3_e","EF3_f","EF3_g","EF3_h"]

# compare the value of "Yes" to EF2 with other EF3 seires all saying "No"
bad_rows = df_shed24[
    (df_shed24["EF2"] == "Yes") &
    (df_shed24[ef3_cols].isin(cannot_pay_values).all(axis=1))
]

bad_rows

,C2A,C4A,EF1,EF2,EF3_a,EF3_b,EF3_c,EF3_d,EF3_e,EF3_f,EF3_g,EF3_h,risk_score,proxy_credit_group
2352,Yes,Never carried an unpaid balance (always pay in...,No,Yes,No,No,No,No,No,No,No,No,1,Super-prime
2877,Yes,Once,No,Yes,No,No,No,No,No,No,No,No,2,Prime
7073,No,,No,Yes,No,No,No,No,No,No,No,No,2,Prime


**We will drop those three 3 rows of people who participated the survey.**

In [ ]:
df_shed24 = df_shed24.drop(index=[2352, 2877, 7073])

Now we will check features **C2A** who has no credit cards comparing with **C4A** who answered how they deal with credit card payments.

In [ ]:
unpaid_categories = {
    "Never carried an unpaid balance (always pay in full)",
    "Once",
    "Some of the time",
    "Most or all of the time"
}

semantic_c4a_error = df_shed24[
    (df_shed24["C2A"] == "No") &
    (df_shed24["C4A"].isin(unpaid_categories))
]

semantic_c4a_error

,C2A,C4A,EF1,EF2,EF3_a,EF3_b,EF3_c,EF3_d,EF3_e,EF3_f,EF3_g,EF3_h,risk_score,proxy_credit_group


There is no error for who is not holding the credit cards answering how they deal with credit card payment.


## Syntactic Check

We will go through all the features where it has the error of the format of answering the survey. We could find implicit missing values during this process.

In [ ]:
binary_cols = ["C2A","EF1","EF2","EF3_a","EF3_b","EF3_c","EF3_d",
               "EF3_e","EF3_f","EF3_g","EF3_h"]

# binary features should answer either yes or no
valid_yes_no = {"Yes", "No"}

# C4A should answer these format as well.
valid_C4A = {"Never carried an unpaid balance (always pay in full)", "Once", "Some of the time", "Most or all of the time"}

# check if valid_yes_no is not valid to binary columns
for col in binary_cols:
    invalid = df_shed24[~df_shed24[col].isin(valid_yes_no)][col].unique()
    if len(invalid) > 0:
        print(f"Syntactic errors in {col}: {invalid}")


Syntactic errors in EF2: ['']


Every rows are good to go, except the feature of **EF2** involving empty string. We could not identify those features because people might refused to answer its answer, or person who worked on this survey made mistake to recorded it, whether they know the answer or not.

In [ ]:
# measuring percentage of empty string
print(((df_shed24['EF2'] == "").sum()*100) / len(df_shed24))
df_shed24[df_shed24['EF2'] == ""].head()

58.17604946306541


,C2A,C4A,EF1,EF2,EF3_a,EF3_b,EF3_c,EF3_d,EF3_e,EF3_f,EF3_g,EF3_h,risk_score,proxy_credit_group
0,Yes,Never carried an unpaid balance (always pay in...,Yes,,Yes,No,No,No,No,No,No,No,0,Super-prime
1,Yes,Never carried an unpaid balance (always pay in...,Yes,,No,No,Yes,No,No,No,No,No,0,Super-prime
3,Yes,Never carried an unpaid balance (always pay in...,Yes,,Yes,No,No,No,No,No,No,No,0,Super-prime
5,Yes,Never carried an unpaid balance (always pay in...,Yes,,Yes,No,No,No,No,No,No,No,0,Super-prime
7,Yes,Never carried an unpaid balance (always pay in...,Yes,,Yes,No,Yes,No,Yes,No,No,No,1,Super-prime


There is over 50% of implicit missing values in **EF2**, we will drop this features due to maintain our representative instead of dropping those rows.

In [ ]:
df_shed24=df_shed24.drop(["EF2"], axis=1)

Let's check the last feature of **C4A** feature.

In [ ]:
invalid_c4a = df_shed24[~df_shed24["C4A"].isin(valid_C4A)]["C4A"].unique()
invalid_c4a

array([''], dtype=object)

In [ ]:
print((df_shed24['C4A'] == "").sum()*100 / len(df_shed24))
df_shed24[df_shed24['C4A'] == ""]

15.823299707126587


,C2A,C4A,EF1,EF3_a,EF3_b,EF3_c,EF3_d,EF3_e,EF3_f,EF3_g,EF3_h,risk_score,proxy_credit_group
73,No,,No,No,No,No,No,No,No,No,Yes,3,Prime
87,No,,Yes,No,No,Yes,No,No,No,No,No,1,Super-prime
104,No,,No,No,Yes,Yes,Yes,No,No,No,No,5,Near-prime
115,No,,Yes,Yes,No,No,No,No,No,No,No,1,Super-prime
135,No,,Yes,No,No,Yes,No,No,No,No,No,1,Super-prime
173,No,,No,No,No,No,No,No,No,No,Yes,3,Prime
180,No,,No,No,No,No,No,Yes,No,No,No,4,Near-prime
183,No,,No,No,No,No,No,Yes,Yes,No,Yes,4,Near-prime
185,No,,No,No,No,Yes,No,No,No,No,No,2,Prime
191,No,,Yes,No,No,Yes,No,No,No,No,No,1,Super-prime


There is 15% of implicit missing values. fter checking the **C2A** feature that has no credit card ownership, **C4A** shows all missing values. This indicates that people did not response due to no ownership. We will change these values as **"No credit card ownership"**.

In [ ]:
df_shed24["C4A"] = df_shed24["C4A"].replace("", "No credit card ownership")
df_shed24["C4A"].unique()

array(['Never carried an unpaid balance (always pay in full)',
       'Most or all of the time', 'Some of the time', 'Once',
       'No credit card ownership'], dtype=object)

In [ ]:
row_df_shed24=row_df_shed24 - df_shed24.shape[0]
row_df_shed24

3

We dropped total 3 rows of SHED 2024 dataset.

We will do same way with **SHED 2020** dataset.

In [ ]:
df_shed20 = df_shed20[["C2A", "C4A","EF1", "EF2","EF3_A","EF3_B","EF3_C","EF3_D",
                       "EF3_E","EF3_F","EF3_G","EF3_H"]]

row_df_shed20 = df_shed20.shape[0]
df_shed20.head()

,C2A,C4A,EF1,EF2,EF3_A,EF3_B,EF3_C,EF3_D,EF3_E,EF3_F,EF3_G,EF3_H
0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,-1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We will use the same method what we did above SHED 2024 dataset using the same features to caculate the risk score to distribute the credit score group, then we will drop the feature of **EF2**.

In [ ]:
df_shed20["risk_score"] = 0

# EF1, EF2: no emergency savings → risk
df_shed20.loc[df_shed20["EF1"] == 0, "risk_score"] += 1
df_shed20.loc[df_shed20["EF2"] == 0, "risk_score"] += 1

# EF3 emergency handling (risky behaviors)
df_shed20.loc[df_shed20["EF3_B"] == 1, "risk_score"] += 1
df_shed20.loc[df_shed20["EF3_E"] == 1, "risk_score"] += 1
df_shed20.loc[df_shed20["EF3_D"] == 1, "risk_score"] += 1

# credit card access
df_shed20.loc[df_shed20["C2A"] == 0, "risk_score"] += 1

# unpaid balance
df_shed20.loc[df_shed20["C4A"] ==1 , "risk_score"] += 1
df_shed20.loc[df_shed20["C4A"] ==2 , "risk_score"] += 2
df_shed20.loc[df_shed20["C4A"] ==3 , "risk_score"] += 3

In [ ]:
def map_group(score):
    if score <= 1:
        return "Super-prime"
    elif score <= 3:
        return "Prime"
    elif score <= 5:
        return "Near-prime"
    elif score <= 7:
        return "Subprime"
    else:
        return "Deep subprime"

df_shed20["proxy_credit_group"] = df_shed20["risk_score"].apply(map_group)

In [ ]:
df_shed20 = df_shed20.drop(["EF2"], axis=1)
df_shed20.head()

,C2A,C4A,EF1,EF3_A,EF3_B,EF3_C,EF3_D,EF3_E,EF3_F,EF3_G,EF3_H,risk_score,proxy_credit_group
0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3,Prime
1,1.0,0.0,-1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,Super-prime
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,Super-prime
3,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Super-prime
4,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Super-prime


This has different format using 0.0 as No, 1.0 as Yes, and -1 as refuse to response. For the feature of **C4A** is 3 as most or all of the time, 2 as some of the time, 1 as once, 0 as never carried an umpaid balance (always pay in full), and -1 as refuse to response.

In [ ]:
df_shed20.isna().sum()

,0
C2A,0
C4A,1605
EF1,0
EF3_A,0
EF3_B,0
EF3_C,0
EF3_D,0
EF3_E,0
EF3_F,0
EF3_G,0


In [ ]:
df_shed20["C4A"].isna().sum()*100 / len(df_shed20)

np.float64(13.77918956043956)

In [ ]:
df_shed20["C4A"].unique()

array([ 1.,  0.,  3., nan,  2., -1.])

In [ ]:
df_shed20[df_shed20["C4A"].isna()]

,C2A,C4A,EF1,EF3_A,EF3_B,EF3_C,EF3_D,EF3_E,EF3_F,EF3_G,EF3_H,risk_score,proxy_credit_group
19,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3,Prime
22,0.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,Super-prime
28,0.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1,Super-prime
50,0.0,NaN,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2,Prime
53,0.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,Super-prime
81,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3,Prime
101,0.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4,Near-prime
109,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,Prime
115,0.0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4,Near-prime
118,0.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,Super-prime


There are quite missing values in **C4A**. It is same as shed 2024 dataset, and we will do the same way what we did above, but we will treated as -2.

In [ ]:
df_shed20["C4A"] = df_shed20["C4A"].fillna(-2)
df_shed20["C4A"].unique()

array([ 1.,  0.,  3., -2.,  2., -1.])

We will also want to check how many people respond the refuse as -1

In [ ]:
df_shed20.eq(-1.0).sum()

,0
C2A,28
C4A,22
EF1,43
EF3_A,65
EF3_B,65
EF3_C,65
EF3_D,65
EF3_E,65
EF3_F,65
EF3_G,65


We have less than we expected. We will drop those -1.0 as well.

In [ ]:
df_shed20 = df_shed20[~df_shed20.eq(-1).any(axis=1)]

df_shed20.eq(-1).sum()

,0
C2A,0
C4A,0
EF1,0
EF3_A,0
EF3_B,0
EF3_C,0
EF3_D,0
EF3_E,0
EF3_F,0
EF3_G,0


Let's check the semantic accuracy on the binary features.

In [ ]:
cannot_pay_values = {"0.0"}

ef3_cols = ["EF1","EF3_A","EF3_B","EF3_C","EF3_D","EF3_E","EF3_F","EF3_G","EF3_H"]

# compare the value of "Yes" to EF2 with other EF3 seires all saying "No"
bad_rows = df_shed20[
    (df_shed20["EF1"] == "1.0") &
    (df_shed20[ef3_cols].isin(cannot_pay_values).all(axis=1))
]

bad_rows

,C2A,C4A,EF1,EF3_A,EF3_B,EF3_C,EF3_D,EF3_E,EF3_F,EF3_G,EF3_H,risk_score,proxy_credit_group


There is no error. Let's move on to **C4A**.

In [ ]:
unpaid_categories = {
    "0.0",
    "1.0",
    "2.0",
    "3.0"
}

# compare the resonse of C2A and C4A
semantic_c4a_error = df_shed20[
    (df_shed20["C2A"] == "0.0") &
    (df_shed20["C4A"].isin(unpaid_categories))
]

semantic_c4a_error

,C2A,C4A,EF1,EF3_A,EF3_B,EF3_C,EF3_D,EF3_E,EF3_F,EF3_G,EF3_H,risk_score,proxy_credit_group


There is no error as well. Let's move on the syntactic accuracy to check the format.

In [ ]:
binary_cols = ["C2A","EF1","EF3_A","EF3_B","EF3_C","EF3_D","EF3_E",
               "EF3_F","EF3_G","EF3_H"]

# binary features should answer either 1 or 0
valid_yes_no = {"1.0", "0.0"}

# C4A should answer these format as well.
valid_C4A = {"0.0", "1.0", "2.0", "3.0"}

# check if valid_yes_no is not valid to binary columns
for col in binary_cols:
    invalid = df_shed20[~df_shed20[col].isin(valid_yes_no)][col].unique()
    if len(invalid) > 0:
        print(f"Syntactic errors in {col}: {invalid}")

Syntactic errors in C2A: [1. 0.]
Syntactic errors in EF1: [0. 1.]
Syntactic errors in EF3_A: [0. 1.]
Syntactic errors in EF3_B: [0. 1.]
Syntactic errors in EF3_C: [1. 0.]
Syntactic errors in EF3_D: [0. 1.]
Syntactic errors in EF3_E: [0. 1.]
Syntactic errors in EF3_F: [0. 1.]
Syntactic errors in EF3_G: [0. 1.]
Syntactic errors in EF3_H: [0. 1.]


There is syntax error as 1. and 0. that we expected to see 1.0 or 0.0. We will make it all consistently integer of 0 or 1.

In [ ]:
df_shed20[binary_cols] = df_shed20[binary_cols].astype(float).astype("Int64")


In [ ]:
valid_yes_no = {1, 0}

#check again after modifying syntax into integer
for col in binary_cols:
    invalid = df_shed20[~df_shed20[col].isin(valid_yes_no)][col].unique()
    if len(invalid) > 0:
        print(f"Syntactic errors in {col}: {invalid}")

Now we don't have error for syntax in binary features. Let's check the **C4A** feature.

In [ ]:
invalid_c4a = df_shed20[~df_shed20["C4A"].isin(valid_C4A)]["C4A"].unique()
invalid_c4a

array([ 1.,  0.,  3., -2.,  2.])

We will make it into integer as well.

In [ ]:
# C4A should answer these format.
valid_C4A = {-2, 0, 1, 2, 3}

df_shed20["C4A"] = df_shed20["C4A"].astype("Int64")

#check again after modifying syntax into integer
invalid_c4a = df_shed20[~df_shed20["C4A"].isin(valid_C4A)]["C4A"].unique()
invalid_c4a

<IntegerArray>
[]
Length: 0, dtype: Int64

There is no syntactic error as well.

In [ ]:
row_df_shed20 = row_df_shed20 - df_shed20.shape[0]
row_df_shed20

137

**We dropped 137 rows total for SHED 2020 dataset.**

Now let's check CFPB dataset in terms of data quality

In [ ]:
print(df_crc.shape)
df_crc

(1100, 5)


,month,date,vol,vol_unadj,credit_score_group
0,84,2007-01,2.544778e+10,2.261068e+10,Super-prime
1,84,2007-01,2.566262e+09,2.139431e+09,Near-prime
2,84,2007-01,4.680974e+08,4.052789e+08,Deep subprime
3,84,2007-01,8.426960e+09,7.433527e+09,Prime
4,84,2007-01,8.445496e+08,7.369389e+08,Subprime
5,85,2007-02,2.436459e+09,1.961025e+09,Near-prime
6,85,2007-02,2.898610e+10,2.508322e+10,Super-prime
7,85,2007-02,4.671905e+08,3.941545e+08,Deep subprime
8,85,2007-02,7.822905e+08,6.413582e+08,Subprime
9,85,2007-02,8.324088e+09,7.269028e+09,Prime


There are 1100 rows and 5 columns. The rows represents how much volumes of credit card released from the bank in each group of credit score. The each feature represents:

- month - Months obervation count from 2000 Jan.
- date - Observation date
- vol - Clean up version of vol_unadj that remove seasonal effects.
- vol_unadj -The actual total dollar volumn of new credit cards opened that month, which the actual money the bank gave out.
- credit_score_group - Divided each group by the credit score as:

  - Deep subprime : <580
  - Subprime : 580 ~ 619
  - Near-prime : 620 ~ 659
  - Prime : 660 ~ 719
  - Super-prime : 720+
So the more closest to the low credit score, the more risk borrower for the lenders. Additionally, the **vol** and **vol_unadj** are similar value.  

Let's check if there is explicit missing values.

In [ ]:
df_crc.isna().sum()

,0
month,0
date,0
vol,0
vol_unadj,0
credit_score_group,0


There is no missing values. How about implicit missing values considering accuracy of semanic and syntactic.

In [ ]:
df_crc.dtypes

,0
month,int64
date,object
vol,float64
vol_unadj,float64
credit_score_group,object


The feature of **month**, **vol**, and **vol_unadj** does not have implicit missing values that showing int64 or float64. Let's check the date and credit_score_group.

In [ ]:
df_crc["date"].value_counts()

,count
date,
2007-01,5
2007-02,5
2007-03,5
2007-04,5
2007-05,5
2007-06,5
2007-07,5
2007-08,5
2007-09,5


The **date** is no implicit missing values and the syntatic and semtic format is good as well.

In [ ]:
df_crc["credit_score_group"].value_counts()

,count
credit_score_group,
Super-prime,220
Near-prime,220
Deep subprime,220
Prime,220
Subprime,220


**credit_score_group** does not have implicit missing value, and the format of semantic and syntatic is good as well.

In [ ]:
# checking if there is negative values on vol and vol_unadj
(df_crc[["vol", "vol_unadj"]] < 0).any()

,0
vol,False
vol_unadj,False


As checking semantic accuracy on **vol** and **vol_unadj**, there is no negative values.

In [ ]:
#checking the flow of the dataset in certain date
df_crc[df_crc["date"] == "2025-01"].sort_values("credit_score_group")

,month,date,vol,vol_unadj,credit_score_group
1082,300,2025-01,4.654316e+08,4.252424e+08,Deep subprime
1080,300,2025-01,1.437533e+09,1.270676e+09,Near-prime
1084,300,2025-01,5.953640e+09,5.565891e+09,Prime
1083,300,2025-01,5.710874e+08,5.182246e+08,Subprime
1081,300,2025-01,3.207861e+10,3.173295e+10,Super-prime


If we check the flow how does credit group issued with volumne in specific date. The semantic accuracy is consistant that bank distributed its volumne in correct format by credit score group as I mentioned earlier.

In [ ]:
df_shed20.to_csv("shed20_cleaned2.csv", index=False)

In [ ]:
df_shed24.to_csv("shed24_cleaned2.csv", index=False)
df_crc.to_csv("cfpd_cleaned2.csv", index=False)

We can download these cleaned datasets from the files